#### Question 5
---

In [ ]:
import numpy as np


cnt = []
w, h = 48, 24

for n, m in [(1,2), (1,3), (2,3)]:
    x = np.arange(1, w // m + 1)
    y = np.arange(1, h // n + 1)
    
    n_x = w - m * x[:, None] + 1
    n_y = h - n * y + 1
    
    cnt.append(np.sum(n_x * n_y))

sum(cnt)

#### Question 6
---

In [ ]:
import numpy as np


def sigmoid(z):
    
    return 1. / (1. + np.exp(-z))


X = np.array([.3,.2,.1,.4,.5])
W_1 = np.array([
    [.21,.22,.31,.42,.32],
    [.32,.41,.42,.42,.52],
])
W_2 = np.array([
    [.3,.2],
    [.3,.14],
])
B_1 = np.array([.21,.3])
B_2 = np.array([.25,.5])

A_1 = sigmoid(np.dot(X, W_1.T) + B_1)
A_2 = sigmoid(np.dot(A_1, W_2.T) + B_2)

sum(A_2)

#### Question 7
---

In [ ]:
import numpy as np


def sigmoid(z):
    
    return 1. / (1. + np.exp(-z))


lr = .6
X = np.array([.5,.8,.7])
W_h_1 = np.array([.24,.43,.52])
W_o_1 = np.array([.12,.42,.1])
b_h_1 = .33
S = np.array([.3,.2,.1])

a_h_1 = sigmoid(np.dot(X, W_h_1) + b_h_1)
dldw = np.dot(S, W_o_1) * (a_h_1 * (1-a_h_1)) * X[0]

W_h_1[0] - lr * dldw

#### Question 7
---

```matlab
A=[
    11 22
    23 18
];

B=[
    4 6
    9 8
];

sum(sum(conv2(A,B)));
```

#### Question 14
---

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier


class AdaBoost:
    def __init__(self, n_clf=2):
        self.n_clf = n_clf
    
    def fit(self, X, y, verbose=False):
        n_samples, n_features = X.shape
        w = np.ones(n_samples) / n_samples
        
        for t in range(self.n_clf):
            model = DecisionTreeClassifier(max_depth=1)
            model.fit(X, y, sample_weight=w)
            y_hat = model.predict(X)
            
            miss = (y_hat != y).astype(int)
            err = np.dot(w, miss)
            
            alpha = 0.5 * np.log((1 - err) / err)
            
            w *= np.exp(-alpha * y * y_hat)
            w /= np.sum(w)
            
            if verbose:
                print(f"t={t}: alpha={alpha:.3f}")
                print(f"data weights={w}")
                print("="*10)
        
        return w


X = np.array([
    [16,38],
    [2,54],
    [32,4],
    [42,10],
    [30,42],
    [22,39],
    [4,33],
    [-22,-25],
    [-37,-31],
    [-23,-48],
])
y = np.array([-1,-1,-1,-1,-1,1,1,1,1,1])

weights = AdaBoost().fit(X, y)
weights.max() - weights.min()

#### Question 16
---

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10))

model.count_params()

#### Question 17
---

In [ ]:
import numpy as np
import keras
from keras import layers


base_model = keras.applications.Xception(
    weights="imagenet",
    input_shape=(150, 150, 3),
    include_top=False,
)

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

x1 = model.count_params()
x2 = sum(sum(var.shape.num_elements() for var in layer.trainable_weights) for layer in model.layers)

base_model.trainable = True

x3 = model.count_params()
x4 = sum(sum(var.shape.num_elements() for var in layer.trainable_weights) for layer in model.layers)

x1 + x2 + x3 + x4